In [ ]:
%matplotlib inline
import chess
import time
import pylab as pl
from IPython import display
import pickle

from bot.bots import HaiBotLong
from bot.model import VFA

### import model

In [ ]:
def player_play(board): 
    your_move = None
    while your_move is None: 
        try: 
            your_move = board.parse_uci(input("uci-move:"))
        except: 
            your_move = board.parse_uci(input("invalid move, need valid uci-move:"))
    board.push(your_move)
    outcome = board.outcome()
    return outcome

def bot_play_wrapper(bot, e=0.02): 
    def bot_play(board): 
        action_bot = bot.play(board, e=e, training=False)
        move_bot = board.parse_uci(action_bot)
        board.push(move_bot)
        outcome = board.outcome()
        return outcome
    return bot_play

### player game

In [ ]:
board = chess.Board()

side = input("choose you side: w for white and b for black")
opponent = "w" if (side == "b") else "b"

model_path = "out/models/cloud2/m_30_iter_3371.ckpt"
bot = HaiBotLong(color=opponent, model_path=model_path)

if side == "w": 
    p1 = player_play
    p2 = bot_play_wrapper(bot)
else: 
    p1 = bot_play_wrapper(bot)
    p2 = player_play
    
    
display.display(board)
while True: 
    outcome = p1(board)
    display.clear_output(wait=True)
    display.display(board)
    if outcome is not None:
        break
        
    time.sleep(0.5)
    outcome = p2(board)
    display.clear_output(wait=True)
    display.display(board)
    if outcome is not None:
        break
print(outcome)

### bot game

In [ ]:
board = chess.Board()

bot1_path = "out/models/champs/m_410_iter_35396.ckpt"
bot2_path = "out/models/cloud2/m_940_iter_68773.ckpt"
bot1 = HaiBotLong(color="W", model_path=bot1_path)
bot2 = HaiBotLong(color="B", model_path=bot2_path)

p1 = bot_play_wrapper(bot1)
p2 = bot_play_wrapper(bot2)
    
display.display(board)
while True: 
    outcome = p1(board)
    display.clear_output(wait=True)
    display.display(board)
    if outcome is not None:
        break
        
    # time.sleep(0.2)
    outcome = p2(board)
    display.clear_output(wait=True)
    display.display(board)
    if outcome is not None:
        break
        
print(outcome)
if outcome.winner is not None: 
    winner = bot1_path if outcome.winner else bot2_path
    print("winner: ", winner)